# 10.1 MNIST

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
learning_rate=0.001
total_epoch=30
batch_size=128

n_input=28
n_step=28
n_hidden=128
n_class=10

X=tf.placeholder(tf.float32,[None,n_step, n_input])
Y=tf.placeholder(tf.float32,[None,n_class])

W=tf.Variable(tf.random_normal([n_hidden,n_class]))
b=tf.Variable(tf.random_normal([n_class]))
cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)

outputs,states=tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
outputs=tf.transpose(outputs,[1,0,2])
outputs=outputs[-1]
model=tf.matmul(outputs,W)+b

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X 데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환합니다.
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy,feed_dict={X: test_xs, Y: test_ys}))

Epoch: 0001 Avg. cost = 0.558
Epoch: 0002 Avg. cost = 0.226
Epoch: 0003 Avg. cost = 0.170
Epoch: 0004 Avg. cost = 0.148
Epoch: 0005 Avg. cost = 0.134
Epoch: 0006 Avg. cost = 0.120
Epoch: 0007 Avg. cost = 0.113
Epoch: 0008 Avg. cost = 0.107
Epoch: 0009 Avg. cost = 0.099
Epoch: 0010 Avg. cost = 0.093
Epoch: 0011 Avg. cost = 0.096
Epoch: 0012 Avg. cost = 0.090
Epoch: 0013 Avg. cost = 0.092
Epoch: 0014 Avg. cost = 0.085
Epoch: 0015 Avg. cost = 0.076
Epoch: 0016 Avg. cost = 0.077
Epoch: 0017 Avg. cost = 0.082
Epoch: 0018 Avg. cost = 0.072
Epoch: 0019 Avg. cost = 0.074
Epoch: 0020 Avg. cost = 0.074
Epoch: 0021 Avg. cost = 0.071
Epoch: 0022 Avg. cost = 0.066
Epoch: 0023 Avg. cost = 0.071
Epoch: 0024 Avg. cost = 0.063
Epoch: 0025 Avg. cost = 0.066
Epoch: 0026 Avg. cost = 0.058
Epoch: 0027 Avg. cost = 0.060
Epoch: 0028 Avg. cost = 0.073
Epoch: 0029 Avg. cost = 0.057
Epoch: 0030 Avg. cost = 0.062
정확도: 0.9713


# 10.2 AutoWordComplete

In [1]:
import tensorflow as tf
import numpy as np

char_arr=[chr(i) for i in range(ord('a'), ord('z')+1)]

num_dic={n:i for i,n in enumerate(char_arr)}
dic_len=len(num_dic)

seq_data=['word','wood','doop','dive','cold', 'cool', 'load', 'love', 'kiss', 'kind']

def make_batch(seq_data):
    input_batch=[]
    target_batch=[]
    
    for seq in seq_data:
        input=[num_dic[n] for n in seq[:-1]]
        target=num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch,target_batch

In [2]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

X=tf.placeholder(tf.float32,[None,n_step,n_input])
Y=tf.placeholder(tf.int32, [None])

W=tf.Variable(tf.random_normal([n_hidden, n_class]))
b=tf.Variable(tf.random_normal([n_class]))

cell1=tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1=tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2=tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell=tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])

outputs,states=tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

outputs=tf.transpose(outputs,[1,0,2])
outputs=outputs[-1]
model=tf.matmul(outputs,W)+b

cost=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

Epoch: 0001 cost = 3.638918
Epoch: 0002 cost = 2.702821
Epoch: 0003 cost = 1.635561
Epoch: 0004 cost = 1.274975
Epoch: 0005 cost = 0.910665
Epoch: 0006 cost = 0.624336
Epoch: 0007 cost = 0.555362
Epoch: 0008 cost = 0.422168
Epoch: 0009 cost = 0.218087
Epoch: 0010 cost = 0.244836
Epoch: 0011 cost = 0.376875
Epoch: 0012 cost = 0.521310
Epoch: 0013 cost = 0.370061
Epoch: 0014 cost = 0.066090
Epoch: 0015 cost = 0.268151
Epoch: 0016 cost = 0.287283
Epoch: 0017 cost = 0.149890
Epoch: 0018 cost = 0.016284
Epoch: 0019 cost = 0.087698
Epoch: 0020 cost = 0.005883
Epoch: 0021 cost = 0.011586
Epoch: 0022 cost = 0.110719
Epoch: 0023 cost = 0.052522
Epoch: 0024 cost = 0.020814
Epoch: 0025 cost = 0.163787
Epoch: 0026 cost = 0.033002
Epoch: 0027 cost = 0.046358
Epoch: 0028 cost = 0.145351
Epoch: 0029 cost = 0.020019
Epoch: 0030 cost = 0.221079


In [4]:
prediction=tf.cast(tf.argmax(model,1), tf.int32)
prediction_check=tf.equal(prediction,Y)
accuracy=tf.reduce_mean(tf.cast(prediction_check, tf.float32))
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words=[]
for idx,val in enumerate(seq_data):
    last_char=char_arr[predict[idx]]
    predict_words.append(val[:3]+last_char)
    
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

입력값: ['wor ', 'woo ', 'doo ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'doop', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
